In [6]:
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import numpy as np
import pickle

In [7]:
stemmer = SnowballStemmer("english")
df = pd.read_csv('clerkie_questions.csv', names = ["class","sentence"])
training_data = df.values
data = df[['sentence']].values
label = df[['class']].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)

In [9]:
#training_data = np.concatenate((y_train, X_train), axis=1)
words = []
classes = []
documents = []
ignore_words = set(stopwords.words('english'))
ignore_words.add('?')
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern[1])
    # add to words list
    words.extend(w)
    # add to documents in corpus
    documents.append((w, pattern[0]))
    # add to classes list
    if pattern[0] not in classes:
        classes.append(pattern[0])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words and not w[0].isdigit()]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

In [10]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)
for doc in documents:
    # initialize bag of words(bow) for each document in the list
    bow = []
    # list of tokenized words for the pattern
    token_words = doc[0]
    # stem each word
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # create bag of words array
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # training set will contain a the bag of words model and the output row that tells
    # which catefory that bow belongs to.
    training.append([bow, output_row])

# shuffle features and turn into np.array as tensorflow  takes in numpy array
random.shuffle(training)
training = np.array(training)


with open('data_label.pickle','wb') as f:
    pickle.dump(training, f)
with open('word_token.pickle','wb') as f:
    pickle.dump(words, f)


